## 2.2) Setting up the foreground system: foreground database

This notebook contains the required steps to import the foreground datatabase. The database comes originally as an excel file.

In [1]:
from pathlib import Path
from pprint import pprint

In [2]:
## Import BW25 packages
import bw2data as bd
import bw2io as bi

In [3]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name

In [4]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

## Transform the data from xlsx format to Brightway format

### Extract foreground inventory data

In [6]:
# Include the path to the foreground database (excel file)
FG_DB = Path("./Foreground.xlsx")

# Import your LCI
lci = bi.ExcelImporter(FG_DB)

Extracted 1 worksheets in 0.04 seconds


### Match the foreground database to itself

In [7]:
# Need to match FG_DB to itself
lci.match_database(fields=["name", "unit", "location"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the background db

In [8]:
# Need to match FG_DB to the biosphere
lci.match_database(ei_name, fields=["name", "reference product", "location", "unit"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the biosphere3 database

In [9]:
# Need to match FG_DB to the biosphere
lci.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields


In [10]:
bi.create_core_migrations()
# Once your package is imported we need to apply strategies
lci.apply_strategies()

# We need to match databases - name and categories but ATTENTION! the categories in
# the excel file is "None" so we will lci.write_excel() also need to match against unit.

lci.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 4.91 seconds
27 datasets
	127 exchanges
	0 unlinked exchanges (0 unique)
	


(27, 127, 0, 0)

In [11]:
import_statistics = lci.statistics(print_stats=False)

# If all exchanges have been linked, proceed to write the database
if import_statistics[2] == 0:
    lci.write_database()
else:
    print(f"There are unlinked exchanges {import_statistics}")
    lci.write_excel()
    for unlinked_exchange in lci.unlinked:
        pprint(unlinked_exchange)

bd.databases

100%|█████████████████████████████████████████| 27/27 [00:00<00:00, 1698.66it/s]


Vacuuming database 
Created database: asphalt


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

### DB structure checks

In [12]:
import bw2analyzer as bwa

/Users/andreavargasf/anaconda3/envs/STI_environment/lib/python3.11/site-packages/bw2calc/__init__.py:48: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


Let's define and assign the name of the foreground database to a variable called "fg_name"
The name must be equal to the content of the cell B2 in the excel file "Foreground.xlsx"

In [13]:
fg_name = "asphalt"

There are many properties and functions associated with this database object. To access those, let's assign the database to a variable

In [14]:
fgdb = bd.Database(fg_name)

Let's explore the foreground db with the function "random"

In [15]:
fgdb.random()

'asphalt granulate, free of burden' (kilogram, NL, None)

In [16]:
random_fg_activity = fgdb.random()
random_fg_activity

'asphalt, pvi, HDV' (kilometer, NL, None)

We can get the type of the object that was returned from the database:

In [17]:
type(random_fg_activity)

bw2data.backends.proxies.Activity

The type is an activity proxy. Activity proxies allow us to interact with the content of the database.

To see what the activity contains, we can convert it to a dictionary:

In [18]:
random_fg_activity.as_dict()

{'reference product': 'extra fuel consumption',
 'code': 'HDV_B',
 'location': 'NL',
 'amount': 1,
 'unit': 'kilometer',
 'original_ConversionDem2FU': 1,
 'name': 'asphalt, pvi, HDV',
 'worksheet name': 'Foreground',
 'database': 'asphalt',
 'id': 25966}

We can get the value of each item by specifying its key. Example: let's get the value corresponding to the key 'location'

In [19]:
random_fg_activity['location']

'NL'

<span style='background:yellow'> Ended here </span> 

In [23]:
pavement = fgdb.get('pavement_structure')

In [24]:
pavement

'pavement structure' (kilometer, NL, None)

In [25]:
bwa.print_recursive_supply_chain(pavement, max_level=1)

1: 'pavement structure' (kilometer, NL, None)
  2.01e+06: 'AC Surf' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)


In [26]:
bwa.print_recursive_supply_chain(pavement, max_level=2)

1: 'pavement structure' (kilometer, NL, None)
  2.01e+06: 'AC Surf' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant' (kilogram, NL, None)
    2.01e+06: 'AC Surf, production' (kilogram, NL, None)
    2.01e+06: 'asphalt, transport to site' (kilogram, NL, None)
    2.01e+06: 'asphalt, construction' (kilogram, NL, None)
    0: 'asphalt, pvi, car' (kilometer, NL, None)
    0: 'asphalt, pvi, HDV' (kilometer, NL, None)
    0: 'asphalt, pvi, trailer' (kilometer, NL, None)
    2.01e+06: 'asphalt, demolition' (kilogram, NL, None)
    0: 'asphalt, transport to processing' (kilogram, NL, None)
    0: 'asphalt, processing' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
    2.72e+06: 'AC Bin, materials and transport to plant' (kilogram, NL, None)
    2.72e+06: 'AC Bin, production' (kilogram, NL, None)
    2.72e+06: 'asphalt, transport to site' (kilogram, NL, None)
    2.72e+06: 'asphalt, construction' (kilogram, NL, None)
    0: 'asphalt, pvi, car' (kilo

In [27]:
bwa.print_recursive_supply_chain(pavement, max_level=3)

1: 'pavement structure' (kilometer, NL, None)
  2.01e+06: 'AC Surf' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant' (kilogram, NL, None)
      9.24e+04: 'bitumen, production and transport' (kilogram, NL, None)
      5.91e+05: 'asphalt granulate, free of burden' (kilogram, NL, None)
      7.35e+05: 'crushed stone, production and transport' (kilogram, NL, None)
      5.18e+05: 'crushed sand, production and transport' (kilogram, NL, None)
      1.81e+04: 'other fillers, production and transport' (kilogram, NL, None)
      5.42e+04: 'weak filler' (kilogram, NL, None)
    2.01e+06: 'AC Surf, production' (kilogram, NL, None)
      1.18e+04: 'market for electricity, low voltage' (kilowatt hour, NL, None)
      5.84e+05: 'heat production, natural gas, at industrial furnace >100kW' (megajoule, Europe without Switzerland, None)
      8.63e+03: 'diesel, burned in building machine' (megajoule, GLO, None)
    2.01e+06: 'asphalt, transport to site' (kilogram, NL, None)

In [28]:
bwa.print_recursive_supply_chain(pavement, max_level=4)

1: 'pavement structure' (kilometer, NL, None)
  2.01e+06: 'AC Surf' (kilogram, NL, None)
    2.01e+06: 'AC Surf, materials and transport to plant' (kilogram, NL, None)
      9.24e+04: 'bitumen, production and transport' (kilogram, NL, None)
        9.24e+04: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
        2.31e+04: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
      5.91e+05: 'asphalt granulate, free of burden' (kilogram, NL, None)
      7.35e+05: 'crushed stone, production and transport' (kilogram, NL, None)
        7.35e+05: 'gravel production, crushed' (kilogram, RoW, None)
        1.84e+04: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
        4.85e+05: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)
      5.18e+05: 'crushed sand, production and transport' (kilogram, NL, None)
        5.18e+05: 'gravel production, crushed' (kilogram, RoW, None)
        1.3